In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
%cd /content/drive/MyDrive/proyectos3/
!ls

/content/drive/MyDrive/proyectos3
alexNet.ipynb  efficientNet.ipynb  GoogleNet.ipynb  PlantDoc		    resNet.ipynb
cnn.ipynb      evaluation	   models	    PlantVillage_bgremoved  wandb


# Cargar Conjunto de Datos

In [21]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import ConcatDataset


train_dir0 = './PlantVillage_bgremoved/train'
train_dir1 = './PlantDoc/train/'

val_dir0 = './PlantVillage_bgremoved/valid'
val_dir1 = './PlantDoc/test/'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset0 = ImageFolder(train_dir0, transform=transform,  is_valid_file=lambda x: x.endswith('.JPG'))
train_dataset1 = ImageFolder(train_dir1, transform=transform)

val_dataset0 = ImageFolder(val_dir0, transform=transform, is_valid_file=lambda x: x.endswith('.JPG'))
val_dataset1 = ImageFolder(val_dir1, transform=transform)

train_dataset = ConcatDataset([train_dataset0, train_dataset1])
val_dataset = ConcatDataset([val_dataset0, val_dataset1])

classes = train_dataset0.classes

classes

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

crear carga de datos para facilitar el acceso a los datos

In [22]:
from torch.utils.data import DataLoader

batch_size = 32
num_workers = 8 # Número de procesos que se utilizarán para cargar datos en paralelo
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [23]:
import torch

def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [24]:
device = get_default_device()

train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

In [25]:
device

device(type='cuda')

In [26]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, f1_score

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module):

    def training_step(self, batch):
        "calculate loss for a batch of training data"
        images, labels = batch
        out = self(images)                  # Generate predictions
        # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        "calculate loss, accuracy, precision and f1 score for a batch of validation data"
        images, labels = batch
        out = self(images)                   # Generate prediction
        # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss
        loss = F.cross_entropy(out, labels)  # Calculate loss
        preds = torch.argmax(out, dim=1)
        acc = accuracy(out, labels)          # Calculate accuracy

        # calculate precision and f1 score
        precision = precision_score(labels.cpu(), preds.cpu(), average='weighted', zero_division=1)
        f1 = f1_score(labels.cpu(), preds.cpu(), average='macro')
        return {'val_loss': loss.detach(), 'val_accuracy': acc, 'val_precision': precision, 'val_f1': f1}

    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()       # Combine loss
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss.item(), "val_accuracy": epoch_accuracy.item()} # Combine accuracies

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_accuracy']))


In [27]:
def BasicConv2d(in_channels, out_channels, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, bias=False, **kwargs),
        nn.BatchNorm2d(out_channels, eps=0.001),
        nn.ReLU(inplace=True)
    )
# Inception module
class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()

        # 1x1 conv branch
        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)

        # 1x1 conv -> 3x3 conv branch
        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1),
            BasicConv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        # 1x1 conv -> 5x5 conv branch
        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1),
            BasicConv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )

        # 3x3 pool -> 1x1 conv branch
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            BasicConv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)

        # concat along channel axis
        return torch.cat([x1, x2, x3, x4], 1)

# GoogLeNet
class GoogLeNet(ImageClassificationBase):
    def __init__(self, num_classes=38):
        super(GoogLeNet, self).__init__()

        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        return x

In [28]:
model = GoogLeNet(len(classes))
model = to_device(model, device)

In [29]:
from torchsummary import summary              # for getting the summary of our model

# getting summary of the model
INPUT_SHAPE = (3, 224, 224)
print(summary(model.cuda(), (INPUT_SHAPE)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8          [-1, 192, 56, 56]         110,592
       BatchNorm2d-9          [-1, 192, 56, 56]             384
             ReLU-10          [-1, 192, 56, 56]               0
        MaxPool2d-11          [-1, 192, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          12,288
      BatchNorm2d-13           [-1, 64, 28, 28]             128
             ReLU-14           [-1, 64,

In [30]:
epochs = 100

max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [31]:
!pip install wandb

In [32]:
import wandb

name = f'googlenet-{epochs}-epochs'
wandb.init(project="plantvillage-aug", name=name, config={
    "epochs": epochs,
    "learning_rate": max_lr,
    "grad_clip": grad_clip,
    "weight_decay": weight_decay,
    "opt_func": opt_func.__name__,
    "batch_size": batch_size,
    "dataset": "PlantVillage",
    "architecture": "GoogleNet"
})

In [33]:
# for training
@torch.no_grad()
def evaluate(model, val_loader):
    """Evaluates the model's performance on the validation set"""
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    # return model.validation_epoch_end(outputs)
    val_losses = [x["val_loss"] for x in outputs]
    epoch_loss = torch.stack(val_losses).mean()       # Combine loss
    val_accuracies = [x["val_accuracy"] for x in outputs]
    epoch_accuracy = torch.stack(val_accuracies).mean()
    val_precisions = [x["val_precision"] for x in outputs]
    epoch_precision = torch.tensor(sum(val_precisions)/len(val_precisions))
    val_f1s = [x["val_f1"] for x in outputs]
    epoch_f1 = torch.tensor(sum(val_f1s)/len(val_f1s))
    return {"val_loss": epoch_loss.item(), "val_accuracy": epoch_accuracy.item(), "val_precision": epoch_precision.item(), "val_f1": epoch_f1.item()} # Combine accuracies


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0,
                grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # scheduler for one cycle learniing rate
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))


    for epoch in range(epochs):
        # Training
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # recording and updating learning rates
            lrs.append(get_lr(optimizer))
            sched.step()

            # logging to wandb
            wandb.log({"Train Loss": loss.item()})


        # validation
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)

        # logging to wandb
        wandb.log({
            "Epoch": epoch,
            "Val Loss": result['val_loss'],
            "Val Accuracy": result['val_accuracy'],
            "Val Precision": result['val_precision'],
            "Val F1": result['val_f1']
        })

    return history

In [34]:
%%time
history = fit_one_cycle(
  epochs,
  max_lr,
  model,
  train_loader,
  val_loader,
  grad_clip=grad_clip,
  weight_decay=1e-4,
  opt_func=opt_func
)

Epoch [0], last_lr: 0.00043, train_loss: 0.4831, val_loss: 0.2910, val_acc: 0.9013
Epoch [1], last_lr: 0.00050, train_loss: 0.2935, val_loss: 0.2884, val_acc: 0.8322
Epoch [2], last_lr: 0.00063, train_loss: 0.2124, val_loss: 0.3034, val_acc: 0.8849
Epoch [3], last_lr: 0.00081, train_loss: 0.2164, val_loss: 0.4166, val_acc: 0.8618
Epoch [4], last_lr: 0.00104, train_loss: 0.2298, val_loss: 0.2098, val_acc: 0.8717
Epoch [5], last_lr: 0.00131, train_loss: 0.2105, val_loss: 0.3847, val_acc: 0.8783
Epoch [6], last_lr: 0.00163, train_loss: 0.1966, val_loss: 0.3343, val_acc: 0.8454
Epoch [7], last_lr: 0.00198, train_loss: 0.2126, val_loss: 0.2328, val_acc: 0.9095
Epoch [8], last_lr: 0.00237, train_loss: 0.2083, val_loss: 0.4430, val_acc: 0.8289
Epoch [9], last_lr: 0.00280, train_loss: 0.1899, val_loss: 0.7965, val_acc: 0.7632
Epoch [10], last_lr: 0.00324, train_loss: 0.2328, val_loss: 1.4000, val_acc: 0.6875
Epoch [11], last_lr: 0.00371, train_loss: 0.2507, val_loss: 0.5188, val_acc: 0.7944
Ep

In [35]:
torch.save(model.state_dict(), f'models/plant_disease_aug_googlenet_{epochs}.pth')

In [36]:
# clear nvidia cache
import torch, gc
gc.collect()
torch.cuda.empty_cache()